# Train smarter and longer

After the original "what am I feeding my network batch size and  video level accuracy. What I've seen is that it should be possible to reach a much higher level. In the com

In [1]:
import os
import sys


up1 = os.path.abspath('../../utils/') 
up2 = os.path.abspath('../../models/') 
sys.path.insert(0, up1)
sys.path.insert(0, up2)

In [2]:
from optical_flow_data_gen import DataGenerator
from ucf101_data_utils import get_test_data_opt_flow, get_train_data_opt_flow
from motion_network import getKerasCifarMotionModel2, getKerasCifarMotionModelOnly
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from keras.optimizers import SGD
import cv2 
import numpy as np
import keras

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


# Is it the data or my classifier

I am starting to wonder what it is about my optical flow data that might be causing so much easier. Regardless about the unconverged flow images I feel the author of the data still managed with it. So there's essentially two things I can either get a large amount of improvement simply on how I train my classifier (slower?), or my data set is not quite right. I've already seen that I wasn't even doing any random transforms on my opt flow images courtesy my badly written opt flow data generator. 
Any how what I am aiming to do is use a stinkingly cheap data model to explore what might be wrong. 

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.initializers import Ones
from keras import optimizers

def getModel(lr=1e-2):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('sigmoid'))

    optimizers.SGD(lr=lr)
    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    return model

In [10]:
model_fast_lr = getModel(lr=1e-2)

In [11]:
from optical_flow_data_gen import DataGenerator
from ucf101_data_utils import get_test_data_opt_flow, get_train_data_opt_flow

training_options = { 'rescale' : 1./255, 
                     'samplewise_center': True, 
                         'samplewise_std_normalization':True,
                            'zoom_range' : 0.2, 
                            'horizontal_flip' : True}

validation_options = { 'rescale' : 1./255,
                     'samplewise_center': True, 
                         'samplewise_std_normalization':True}




params_train = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size': 128,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : False,
                'enable_augmentation' : True,
               'training_opts' : training_options}

params_valid = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size':128,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : True,
               'validation_opts' : validation_options}

id_labels_train = get_train_data_opt_flow('../../data/ucf101_splits/trainlist01_small.txt')
labels = id_labels_train[1]
id_test = get_test_data_opt_flow('../../data/ucf101_splits/testlist01_small.txt', \
                       '../../data/ucf101_splits/classInd_small.txt')

training_generator = DataGenerator(*id_labels_train, **params_train)


validation_generator = DataGenerator(id_test[0], id_test[1], **params_valid)

# With Augmentation

In [12]:
mod1 = model_fast_lr.fit_generator(generator=training_generator, steps_per_epoch=64,
                    validation_data=validation_generator, validation_steps=64,
                    use_multiprocessing=True,
                    workers=2, epochs=5,
                    verbose=1)

Epoch 1/5
64/64 [==============================] - 107s 2s/step - loss: 2.2618 - acc: 0.1385 - val_loss: 2.2421 - val_acc: 0.2026
Epoch 2/5
64/64 [==============================] - 107s 2s/step - loss: 2.2097 - acc: 0.2037 - val_loss: 2.1872 - val_acc: 0.2521
Epoch 3/5
64/64 [==============================] - 99s 2s/step - loss: 2.1595 - acc: 0.2393 - val_loss: 2.1423 - val_acc: 0.2833
Epoch 4/5
64/64 [==============================] - 93s 1s/step - loss: 2.1254 - acc: 0.2654 - val_loss: 2.1007 - val_acc: 0.3154
Epoch 5/5
64/64 [==============================] - 99s 2s/step - loss: 2.0843 - acc: 0.2784 - val_loss: 2.0944 - val_acc: 0.3096


# Without Augmentation


In [7]:
params_train = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size': 128,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : False,
                'enable_augmentation' : False,
               'training_opts' : training_options}

training_generator = DataGenerator(*id_labels_train, **params_train)

model_fast_lr = getModel(lr=1e-2)


In [8]:
mod1 = model_fast_lr.fit_generator(generator=training_generator, steps_per_epoch=64,
                    validation_data=validation_generator, validation_steps=64,
                    use_multiprocessing=True,
                    workers=2, epochs=5,
                    verbose=1)

Epoch 1/5
64/64 [==============================] - 63s 985ms/step - loss: 2.2386 - acc: 0.1510 - val_loss: 2.2132 - val_acc: 0.1897
Epoch 2/5
64/64 [==============================] - 62s 963ms/step - loss: 2.1650 - acc: 0.2238 - val_loss: 2.1443 - val_acc: 0.2300
Epoch 3/5
64/64 [==============================] - 60s 938ms/step - loss: 2.0883 - acc: 0.2601 - val_loss: 2.0870 - val_acc: 0.3040
Epoch 4/5
64/64 [==============================] - 60s 936ms/step - loss: 2.0346 - acc: 0.2836 - val_loss: 2.0424 - val_acc: 0.3174
Epoch 5/5
64/64 [==============================] - 60s 933ms/step - loss: 1.9951 - acc: 0.2933 - val_loss: 2.0482 - val_acc: 0.3444


## Long run with agumentation

The idea here is to run for  180 epochs which should be about  12,000 iterations. But the learning rate will be regulated to try  to maximize convergence

In [13]:
params_train = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size': 128,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : False,
                'enable_augmentation' : True,
               'training_opts' : training_options}

training_generator = DataGenerator(*id_labels_train, **params_train)

model_fast_lr = getModel(lr=1e-2)

In [ ]:
mod1 = model_fast_lr.fit_generator(generator=training_generator, steps_per_epoch=64,
                    validation_data=validation_generator, validation_steps=64,
                    use_multiprocessing=True,
                    workers=2, epochs=60,
                    verbose=1)

Epoch 1/60
64/64 [==============================] - 105s 2s/step - loss: 2.2382 - acc: 0.1262 - val_loss: 2.2333 - val_acc: 0.1855
Epoch 2/60
64/64 [==============================] - 97s 2s/step - loss: 2.1782 - acc: 0.2043 - val_loss: 2.1670 - val_acc: 0.2777
Epoch 3/60
64/64 [==============================] - 93s 1s/step - loss: 2.1403 - acc: 0.2579 - val_loss: 2.1217 - val_acc: 0.3157
Epoch 4/60
64/64 [==============================] - 94s 1s/step - loss: 2.1016 - acc: 0.2795 - val_loss: 2.0963 - val_acc: 0.3297
Epoch 5/60
64/64 [==============================] - 96s 2s/step - loss: 2.0586 - acc: 0.2848 - val_loss: 2.0553 - val_acc: 0.3229
Epoch 6/60
64/64 [==============================] - 95s 1s/step - loss: 2.0290 - acc: 0.2959 - val_loss: 2.0189 - val_acc: 0.3282
Epoch 7/60
64/64 [==============================] - 99s 2s/step - loss: 1.9858 - acc: 0.2991 - val_loss: 1.9977 - val_acc: 0.3181
Epoch 8/60
64/64 [==============================] - 95s 1s/step - loss: 1.9739 - acc: 0.3